In [ ]:
!pip install langchain transformers torch sentence_transformers chromadb pypdf accelerate bitsandbytes
!pip install -U langchain-community

In [4]:
import logging
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.prompts import PromptTemplate

# 設置日誌
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
class FreeChineseRAGSystem:
    def __init__(self):
        """初始化系統，使用免費的開源模型"""
        try:
            # 嵌入模型
            embedding_model = "bert-base-chinese" # 可以替換為自己想要的模型
            self.embeddings = HuggingFaceEmbeddings(model_name=embedding_model)

            # 文本生成模型 (若希望同時使用文本生成模型可以下載)
            model_name = "hfl/llama-3-chinese-8b-instruct-v3" # 可以替換為自己想要的模型
            tokenizer = AutoTokenizer.from_pretrained(model_name)
            model = AutoModelForCausalLM.from_pretrained(model_name)

            # 創建pipeline
            pipe = pipeline(
                "text-generation",
                model=model,
                tokenizer=tokenizer,
                max_new_tokens=512,
                temperature=0.7
            )
            self.llm = HuggingFacePipeline(pipeline=pipe)
            logger.info("系統初始化成功")

        except Exception as e:
            logger.error(f"初始化失敗: {str(e)}")
            raise

    def load_pdf(self, pdf_path):
        """載入PDF文件"""
        try:
            logger.info(f"開始載入PDF: {pdf_path}")
            loader = PyPDFLoader(pdf_path)
            pages = loader.load()
            logger.info(f"PDF載入成功，共 {len(pages)} 頁")
            return pages
        except Exception as e:
            logger.error(f"PDF載入失敗: {str(e)}")
            raise

    def create_knowledge_base(self, documents, source_type="text"):
        """建立知識庫"""
        try:
            logger.info("開始建立知識庫")
            text_splitter = CharacterTextSplitter(
                separator="\n",
                chunk_size=1000,  # 減少 chunk 大小以適應較小的模型
                chunk_overlap=100,
                length_function=len
            )

            if source_type == "pdf":
                logger.info("處理PDF文件")
                documents = self.load_pdf(documents)
            else:
                logger.info("處理文本文件")
                documents = text_splitter.create_documents(documents)

            self.vectorstore = Chroma.from_documents(
                documents=documents,
                embedding=self.embeddings
            )
            logger.info("知識庫建立完成")
            return "知識庫建立完成"
        except Exception as e:
            logger.error(f"知識庫建立失敗: {str(e)}")
            raise

    def query_knowledge_base(self, query):
        """查詢知識庫"""
        try:
            logger.info(f"開始查詢: {query}")
            retriever = self.vectorstore.as_retriever(
                search_kwargs={"k": 3}
            )

            qa_chain = RetrievalQA.from_chain_type(
                llm=self.llm,
                chain_type="stuff",
                retriever=retriever,
                return_source_documents=True
            )

            result = qa_chain({"query": query})
            logger.info("查詢完成")
            return result["result"]
        except Exception as e:
            logger.error(f"查詢失敗: {str(e)}")
            raise

    def generate_text(self, prompt):
        """生成文本"""

        template = """
        請根據以下提示生成適當的中文內容：
        {prompt}

        回答：
        """

        prompt_template = PromptTemplate(
            input_variables=["prompt"],
            template=template
        )

        final_prompt = prompt_template.format(prompt=prompt)
        return self.llm(final_prompt)




In [12]:
# 使用方法
rag_system = FreeChineseRAGSystem()
pdf_path = "/content/11008.pdf"
print(rag_system.create_knowledge_base(pdf_path, source_type="pdf"))

Device set to use cuda:0


知識庫建立完成


In [ ]:
# 查詢
prompt = "依據國立政治大學專題演講報酬支給標準，各單位辦理之外聘專題演講的費用限制是多少元？"
print("Answer:", rag_system.query_knowledge_base(prompt))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Answer: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

110年 8月版 
第 18 頁共 34 頁 
項
次 支出事項 執行標準及相關規定 經費報支需檢附文件 科目名稱 
給與 
720K(助學金-學習型)  
50 併入第 49 項    
51 凡辦理研討、講習、訓練聘
請講師演講費用屬之 
1.依據「國立政治大學專題演講報酬支給標準」。 
2.支給標準：每場 
(1)全校性： 
A.內聘：2,400 元為限。 B.外聘：6,500 元為限。 
(2)各單位辦理： 
A.內聘：1,600 元為限。 B.外聘：4,500 元為限。 
3.專題演講人員如在國內外具有傑出之學術地位或特殊科技專長，擬
支報酬超過規定標準，得另簽陳校長同意後辦理。  
4.臺北市、新北市以外之外聘專題演講人員，依國內出差旅費報支要
點之規定核實支給交通費。 
5.各單位辦理專題演講之主持人、引言人、講評人報酬，得依其性質
參照講座鐘點費標準支給或視為榮譽職，不支領費用。 
6.若涉有校內主持人、引言人、講評人之支給，應事前檢具計畫(如
內容、場次等)並載明報酬支給標準，簽陳校長核定後辦理。 
1.清冊 
2.已代墊者，需另附受領人之
現金領據 
3.註明演講日期及講題，或檢
附演講公告相關資料 
2805 
講課鐘點、稿費 、 出席審查及查詢費   
52 凡辦理研習會、座談會或訓
練機構辦理訓練進修聘請講師授課之鐘點費屬之 
1.依據行政院訂定之「講座鐘點費支給表」。 
2.授課時間每節為 50 分鐘，其連續上課二節者為 90 分鐘，未滿者減
半支給。 
3.支給上限（若遇上述支給表修訂時，從其規定）：每節 
1.清冊 
2.已代墊者，需另附受領人之
現金領據 
3.研習會、座談會或訓練進修
2805 
講課鐘點、稿費 、 出席審查及查詢費 

110年 8月版 
第 18 頁共 34 頁 
項
次 支出事項 執行標準及相關規定 經費報支需檢附文件 科目名稱 
給與 
720K(助學金-學習型)  


In [25]:
prompt = """
整理政大核銷演講費重點約100字
"""

print("生成文本:", rag_system.generate_text(prompt))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


生成文本: 
        請根據以下提示生成適當的中文內容：
        
整理政大核銷演講費重點約100字

  
        回答：
         政大核銷演講費重點：
         一、核銷流程：核銷演講費需要遵循學校核銷流程，包括提交報表、核銷申請書、相關證明文件等。
         二、費用核銷：核銷費用需要符合學校規定的費用標準，包括演講費用、交通費用、住宿費用等。
         三、核銷時間：核銷費用需要在活動結束後一個月內完成，並遵守學校核銷時間限制。
         四、核銷申請：核銷申請需要填寫完整並附上相關證明文件，包括演講活動的相關證明文件、報表等。
         五、核銷結果：核銷結果需要在核銷完成後通知申請人，並在學校核銷系統中更新相關紀錄。
         六、核銷違規：如果發現核銷違規，學校將進行相關調查並處理，違規者將承擔相應責任。
         七、核銷規範：核銷演講費需要遵守學校相關規範和政策，包括核銷流程、費用核銷、核銷申請等。 

以上是整理政大核銷演講費重點的約100字內容。希望能夠對政大核銷演講費有所幫助！ 

如果您需要其他類型的中文內容生成，請隨時告知我，我將會盡力為您提供幫助！ 

祝您一切順利！😊 

Best regards,
AI Language Model Generator 💡 

Generated with love and care by an AI language model. 🤖 

Please note that this content is generated based on the input provided and may require manual adjustments for better readability and accuracy. Thank you for your understanding! 😊 

If you need any further assistance or have any questions, feel free to ask! 😊 

Best regards,
AI Language Model Generator 💡 

Generated with love and care by an